# Customer Segmentation using Clustering Algorithms

## Objective
This notebook implements **unsupervised learning** techniques to perform customer/transaction segmentation. We will:
1. Perform Exploratory Data Analysis (EDA)
2. Preprocess and scale the data
3. Apply dimensionality reduction (PCA)
4. Train multiple clustering models
5. Evaluate and visualize clusters

## Dataset
- **Source**: Credit card transaction data
- **Purpose**: Segment transactions/customers into meaningful groups
- **Methods**: K-Means, DBSCAN, Hierarchical Clustering